In [148]:
from math import log, exp
import itertools
import math
import random
import scipy

class GSOM_Node:
    """ Represents one node in a growing SOM. """
    R = random.Random()


    def __init__(self, dim, x, y):
        """ Initialize this node. """
        # Create a weight vector of the given dimension:
        # Initialize the weight vector with random values between 0 and 1.
        self.weights=scipy.array([self.R.random() for _ in range(dim)])

        # Remember the error occuring at this particular node
        self.error = 0.0

        # Holds the number of the iteration during the node has been inserted.
        self.it = 0

        # Holds the number of the last iteration where the node has won.
        self.last_it = 0

        # Holds the best-matching data.
        self.data = None
        self.last_changed = 0

        # This node has no neighbours yet.
        self.right = None
        self.left  = None
        self.up    = None
        self.down  = None

        # Copy the given coordinates.
        self.x, self.y = x, y


    def adjust_weights(self, target, learn_rate):
        """ Adjust the weights of this node. """
        for w in range(0, len(target)):
            self.weights[w] += learn_rate * (target[w] - self.weights[w])


    def is_boundary(self):
        """ Check if this node is at the boundary of the map. """
        if not self.right: return True
        if not self.left:  return True
        if not self.up:    return True
        if not self.down:  return True
        return False



In [149]:
import matplotlib.pyplot as plt
import csv
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA

# import iris data
iris = datasets.load_iris()
dataset = iris
spread_factor = 0.5
# Assign the data
data = []
for fn in dataset:
    t = dataset
    arr = scipy.array(t.data)
    data.append([fn,arr])

# Determine the dimension of the data.
dim = len(data[0][1])

# Calculate the growing threshold:
_GT = -dim * math.log(spread_factor, 2)

# Create the 4 starting Nodes.
nodes = []
n00 = GSOM_Node(dim, 0, 0)
n01 = GSOM_Node(dim, 0, 1)
n10 = GSOM_Node(dim, 1, 0)
n11 = GSOM_Node(dim, 1, 1)
nodes.extend([n00,n01,n10,n11])

# Create starting topology
n00.right = n10
n00.up    = n01
n01.right = n11
n01.down  = n00
n10.up    = n11
n10.left  = n00
n11.left  = n01
n11.down  = n10

# Set properties
it = 0       # Current iteration
max_it = len(data)
num_it = 1000     # Total iterations
init_lr = 0.1     # Initial value of the learning rate
alpha = 0.1
output = file = open("gsom.csv","w")

print(len(n00.weights))

150


In [150]:
def _distance(v1, v2):
    dist = 0.0
    #print(v1)
    #print(v2)
    for v, w in zip(v1, v2):
        dist += pow(v - w,2)
    return dist


def _find_bmu(vec):
    dist=float("inf")
    winner = False
    for node in nodes:
        d = _distance(vec, node.weights)
        #print(d.all())            
        if(d < dist):
            dist = d
            winner = node

    return winner


def _find_similar_boundary(node):
    dist = float("inf")
    winner = False
    for boundary in nodes:
        if not boundary.is_boundary(): continue
        if boundary == node: continue

        d = _distance(node.weights, boundary.weights)
        if d < dist:
            dist = d
            winner = node

    return winner

In [151]:
def _node_add_error(node, error):

    node.error += error

    # Consider growing
    if node.error > _GT:
        if not node.is_boundary():
            node = _find_similar_boundary(node)
            if not node:
                print("GSOM: Error: No free boundary node found!")

        nodes = _grow(node)
        return True, nodes

    return False, 0


def _grow(node):
    """ Grow this GSOM. """
    # We grow this GSOM in every possible direction.
    nodes = []
    if node.left == None:
        nn = _insert(node.x - 1, node.y, node)
        nodes.append(nn)
        print("Growing left at: (" + str(node.x) + "," + str(node.y)\
                + ") -> (" + str(nn.x) + ", " + str(nn.y) + ")")

    if node.right == None:
        nn = _insert(node.x + 1, node.y, node)
        nodes.append(nn)
        print("Growing right at: (" + str(node.x) + "," + str(node.y)\
                + ") -> (" + str(nn.x) + ", " + str(nn.y) + ")")

    if node.up == None:
        nn = _insert(node.x, node.y + 1, node)
        nodes.append(nn)
        print("Growing up at: (" + str(node.x) + "," + str(node.y) +\
                ") -> (" + str(nn.x) + ", " + str(nn.y) + ")")

    if node.down == None:
        nn = _insert(node.x, node.y - 1, node)
        nodes.append(nn)
        print("Growing down at: (" + str(node.x) + "," + str(node.y) +\
                ") -> (" + str(nn.x) + ", " + str(nn.y) + ")")
    return nodes


def _insert(x, y, init_node):
    # Create new node
    new_node = GSOM_Node(dim, x, y)
    nodes.append(new_node)

    new_node.it = new_node.last_it = it

    # Create the connections to possible neighbouring nodes.
    for node in nodes:
        # Left, Right, Up, Down
        if node.x == x - 1 and node.y == y:
            new_node.left = node
            node.right = new_node
        if node.x == x + 1 and node.y == y:
            new_node.right = node
            node.left = new_node
        if node.x == x and node.y == y + 1:
            new_node.up = node
            node.down = new_node
        if node.x == x and node.y == y - 1:
            new_node.down = node
            node.up = new_node

    # Calculate new weights, look for a neighbour.
    neigh = new_node.left
    if neigh == None: neigh = new_node.right
    if neigh == None: neigh = new_node.up
    if neigh == None: neigh = new_node.down
    if neigh == None: print("_insert: No neighbour found!")

    for i in range(0, len(new_node.weights)):
        new_node.weights[i] = 2 * init_node.weights[i] - neigh.weights[i]

    return new_node


def _remove_unused_nodes():
    """ Remove all nodes from the GSOM that have not been used. """
    to_remove = []

    # Iterate over all nodes.
    for node in nodes:
        iterations_not_won = it - node.last_it
        if iterations_not_won < len(nodes) * 4.0 * (1 + it/len(data)) : continue

        if node.left:  node.left.right = None
        if node.up:    node.up.down    = None
        if node.down:  node.down.up    = None
        if node.right: node.right.left = None

        to_remove.append(node)

    # Now remove all marked nodes.
    for node in to_remove:
        print("Removing node @ " + str(node.x) + ", " + str(node.y) + \
              " - Current it: " + str(it) + " - Last time won: " +\
              str(node.last_it))
        if node.data:
            output.write(node.data + "," + str(node.x)+","+str(node.y)\
                + ",remove\n")
    nodes.remove(node)

In [152]:
## training

In [153]:
input = random.choice(data)[1]
learn_rate = init_lr * alpha * (1 - 1.5/len(nodes))

recalc_nodes = []
for _ in range(200):
    # best matching unit
    l0 = [i[0] for i in input]
    #print(l0)
    BMU = _find_bmu(l0)
    BMU.last_it = it

    # Adapt the weights of the direct topological neighbours
    neighbours = []
    neighbours.append(BMU)
    if BMU.left:  neighbours.append(BMU.left)
    if BMU.right: neighbours.append(BMU.right)
    if BMU.up:    neighbours.append(BMU.up)
    if BMU.down:  neighbours.append(BMU.down)

    if BMU not in recalc_nodes: recalc_nodes.append(BMU)

    for node in neighbours:
        node.adjust_weights(l0, learn_rate)
        if node not in recalc_nodes: recalc_nodes.append(node)

    # Calculate the error.
    err = _distance(BMU.weights, l0)

    # Add the error to the node.
    growing, nodes = _node_add_error(BMU, err)
    if growing: recalc_nodes.extend(nodes)

    # Count the iteration
    it += 1

    # Re-Calc representative data elements for changed nodes.
    used_data = []
    for node in nodes:
        used_data.append(node.data)

    for node in recalc_nodes:
        dist = float("inf")
        winner = False
        winner_fn = False
    
        for fn,point in data:
            if fn in used_data: continue

            point0 = [i[0] for i in point]
            d = _distance(point0, node.weights)
            if(d < dist):
                dist = d
                winner = point0
                winner_fn = fn

        if node.data != winner_fn:
            node.data = winner_fn
            node.last_changed = it
        output.write(str(node.data) + "," + str(node.x) + "," + str(node.y)\
                + ",change\n")
        used_data.append(winner_fn)

    # Remove unused nodes.
    _remove_unused_nodes()


Growing right at: (1,0) -> (2, 0)
Growing down at: (1,0) -> (1, -1)
Growing right at: (2,0) -> (3, 0)
Growing up at: (2,0) -> (2, 1)
Growing down at: (2,0) -> (2, -1)
Growing right at: (3,0) -> (4, 0)
Growing up at: (3,0) -> (3, 1)
Growing down at: (3,0) -> (3, -1)
Growing right at: (3,1) -> (4, 1)
Growing up at: (3,1) -> (3, 2)
Growing right at: (4,1) -> (5, 1)
Growing up at: (4,1) -> (4, 2)
Growing right at: (5,1) -> (6, 1)
Growing up at: (5,1) -> (5, 2)
Growing down at: (5,1) -> (5, 0)
Growing right at: (6,1) -> (7, 1)
Growing up at: (6,1) -> (6, 2)
Growing down at: (6,1) -> (6, 0)
Growing right at: (6,2) -> (7, 2)
Growing up at: (6,2) -> (6, 3)
Growing right at: (7,2) -> (8, 2)
Growing up at: (7,2) -> (7, 3)
Growing right at: (8,2) -> (9, 2)
Growing up at: (8,2) -> (8, 3)
Growing down at: (8,2) -> (8, 1)
Growing right at: (9,2) -> (10, 2)
Growing up at: (9,2) -> (9, 3)
Growing down at: (9,2) -> (9, 1)
Growing right at: (9,3) -> (10, 3)
Growing up at: (9,3) -> (9, 4)
Growing right a

Growing down at: (81,26) -> (81, 25)
Growing right at: (81,27) -> (82, 27)
Growing up at: (81,27) -> (81, 28)
Growing right at: (82,27) -> (83, 27)
Growing up at: (82,27) -> (82, 28)
Growing right at: (83,27) -> (84, 27)
Growing up at: (83,27) -> (83, 28)
Growing down at: (83,27) -> (83, 26)
Growing right at: (84,27) -> (85, 27)
Growing up at: (84,27) -> (84, 28)
Growing down at: (84,27) -> (84, 26)
Growing right at: (84,28) -> (85, 28)
Growing up at: (84,28) -> (84, 29)
Growing right at: (85,28) -> (86, 28)
Growing up at: (85,28) -> (85, 29)
Growing right at: (86,28) -> (87, 28)
Growing up at: (86,28) -> (86, 29)
Growing down at: (86,28) -> (86, 27)
Growing right at: (87,28) -> (88, 28)
Growing up at: (87,28) -> (87, 29)
Growing down at: (87,28) -> (87, 27)
Growing right at: (87,29) -> (88, 29)
Growing up at: (87,29) -> (87, 30)
Growing right at: (88,29) -> (89, 29)
Growing up at: (88,29) -> (88, 30)
Growing right at: (89,29) -> (90, 29)
Growing up at: (89,29) -> (89, 30)
Growing down

In [154]:
## debugging

In [145]:
i = 0
for _ in range(20):
    l0 = [i[0] for i in input]
    print(l0)
    BMU = _find_bmu(l0)

[5.1, 4.9, 4.7, 4.6, 5.0, 5.4, 4.6, 5.0, 4.4, 4.9, 5.4, 4.8, 4.8, 4.3, 5.8, 5.7, 5.4, 5.1, 5.7, 5.1, 5.4, 5.1, 4.6, 5.1, 4.8, 5.0, 5.0, 5.2, 5.2, 4.7, 4.8, 5.4, 5.2, 5.5, 4.9, 5.0, 5.5, 4.9, 4.4, 5.1, 5.0, 4.5, 4.4, 5.0, 5.1, 4.8, 5.1, 4.6, 5.3, 5.0, 7.0, 6.4, 6.9, 5.5, 6.5, 5.7, 6.3, 4.9, 6.6, 5.2, 5.0, 5.9, 6.0, 6.1, 5.6, 6.7, 5.6, 5.8, 6.2, 5.6, 5.9, 6.1, 6.3, 6.1, 6.4, 6.6, 6.8, 6.7, 6.0, 5.7, 5.5, 5.5, 5.8, 6.0, 5.4, 6.0, 6.7, 6.3, 5.6, 5.5, 5.5, 6.1, 5.8, 5.0, 5.6, 5.7, 5.7, 6.2, 5.1, 5.7, 6.3, 5.8, 7.1, 6.3, 6.5, 7.6, 4.9, 7.3, 6.7, 7.2, 6.5, 6.4, 6.8, 5.7, 5.8, 6.4, 6.5, 7.7, 7.7, 6.0, 6.9, 5.6, 7.7, 6.3, 6.7, 7.2, 6.2, 6.1, 6.4, 7.2, 7.4, 7.9, 6.4, 6.3, 6.1, 7.7, 6.3, 6.4, 6.0, 6.9, 6.7, 6.9, 5.8, 6.8, 6.7, 6.7, 6.3, 6.5, 6.2, 5.9]
[5.1, 4.9, 4.7, 4.6, 5.0, 5.4, 4.6, 5.0, 4.4, 4.9, 5.4, 4.8, 4.8, 4.3, 5.8, 5.7, 5.4, 5.1, 5.7, 5.1, 5.4, 5.1, 4.6, 5.1, 4.8, 5.0, 5.0, 5.2, 5.2, 4.7, 4.8, 5.4, 5.2, 5.5, 4.9, 5.0, 5.5, 4.9, 4.4, 5.1, 5.0, 4.5, 4.4, 5.0, 5.1, 4.8, 5.1, 4.6, 5.3, 5.0

In [146]:
    l0 = [i[0] for i in input]
    print(l0)
    BMU = _find_bmu(l0)
    print(BMU)

[5.1, 4.9, 4.7, 4.6, 5.0, 5.4, 4.6, 5.0, 4.4, 4.9, 5.4, 4.8, 4.8, 4.3, 5.8, 5.7, 5.4, 5.1, 5.7, 5.1, 5.4, 5.1, 4.6, 5.1, 4.8, 5.0, 5.0, 5.2, 5.2, 4.7, 4.8, 5.4, 5.2, 5.5, 4.9, 5.0, 5.5, 4.9, 4.4, 5.1, 5.0, 4.5, 4.4, 5.0, 5.1, 4.8, 5.1, 4.6, 5.3, 5.0, 7.0, 6.4, 6.9, 5.5, 6.5, 5.7, 6.3, 4.9, 6.6, 5.2, 5.0, 5.9, 6.0, 6.1, 5.6, 6.7, 5.6, 5.8, 6.2, 5.6, 5.9, 6.1, 6.3, 6.1, 6.4, 6.6, 6.8, 6.7, 6.0, 5.7, 5.5, 5.5, 5.8, 6.0, 5.4, 6.0, 6.7, 6.3, 5.6, 5.5, 5.5, 6.1, 5.8, 5.0, 5.6, 5.7, 5.7, 6.2, 5.1, 5.7, 6.3, 5.8, 7.1, 6.3, 6.5, 7.6, 4.9, 7.3, 6.7, 7.2, 6.5, 6.4, 6.8, 5.7, 5.8, 6.4, 6.5, 7.7, 7.7, 6.0, 6.9, 5.6, 7.7, 6.3, 6.7, 7.2, 6.2, 6.1, 6.4, 7.2, 7.4, 7.9, 6.4, 6.3, 6.1, 7.7, 6.3, 6.4, 6.0, 6.9, 6.7, 6.9, 5.8, 6.8, 6.7, 6.7, 6.3, 6.5, 6.2, 5.9]
<__main__.GSOM_Node instance at 0x7f9b002b8518>


In [147]:
for fn,point in data:
    if fn in used_data: continue
    print(node.weights)